In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')



In [2]:
from config_paper import key_id, secret_key
from cross_signal import *

In [3]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id, secret_key,base_url )

In [4]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-04-28'

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['ema_high'] = tab.EMA(dataframe, timeperiod=10, price='high')
        dataframe['ema_close'] = tab.EMA(dataframe, timeperiod=10, price='close')
        dataframe['ema_low'] = tab.EMA(dataframe, timeperiod=10, price='low')
        
        
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        dataframe['adosc'] = ta.ADOSC(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'))
        
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        stoch_fast = tab.STOCHF(dataframe, 10.0, 3.0, 0.0, 3.0, 0.0)
        dataframe['fastd'] = stoch_fast['fastd']
        dataframe['fastk'] = stoch_fast['fastk']
        dataframe['adx'] = tab.ADX(dataframe)
        dataframe['cci'] = tab.CCI(dataframe)
        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'93286.52'

In [7]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [8]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.10

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)


# Bar get with alpaca

In [ ]:
for minu in range(500) :
        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.get_barset(symbol,barTimeframe,limit=150).df
            #api.get_barset(symbol,barTimeframe,limit=150).df


            # Calculated trading indicators
            #populate exponential moving averages
            data=populateindicators(returned_data[symbol]) 


             # Create the buy signal here
            if data['adx'][-1] > 22 and \
            data['mfi'][-1] < 25 and \
            data['ultosc'][-1] < 25 and \
            data['rsi'][-1] < 33 and \
            data['minusdi'][-1] > 25 and \
            data['minusdi'][-1] > data['plusdi'][-1]:

                buyord=True
                assetlist=api.list_positions()
                for asset in assetlist:
                    if asset.symbol == symbol:
                        buyord=False
                        print(asset.symbol,asset.qty)
                if buyord==True:
                    #cashBalance = account.cash

                    targetPositionSize = int(2000 / data['close'][-1]) # Calculates required position size

                    returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                    print('Buy ',returned)


            elif data['adx'][-1] > 22 and \
            data['mfi'][-1] > 63 and \
            data['rsi'][-1] > 75 and \
            data['plusdi'][-1] > 25 and \
            data['minusdi'][-1] < data['plusdi'][-1]:
                # Closes position
                openPosition = api.get_position(symbol)

                returned = api.submit_order(symbol,int(openPosition.qty),"sell","market","day") # Market order to fully close position
                print('Sell ',returned)
        
        except:
                print(symbol)
                
        #interval should be an integer, the number of seconds to wait
    time.sleep(20)

C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '419e89a6-efe2-4773-ba4f-1bf5dca33cb2',
    'canceled_at': None,
    'client_order_id': 'cc0aeaf6-4498-4be3-9698-7b07c68214e7',
    'created_at': '2020-04-28T15:59:11.271329Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'b5f015d0-f699-4f2e-923c-37495604c219',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '7',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-04-28T15:59:11.254056Z',
    'symbol': 'BDX',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-04-28T15:59:11.271329Z'})
BRKB
BFB
CMS 34
DISCA
DG 11
ES 23
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '09192535-21d6-42cf-903d-4abd88f91cad',
    'c

sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1Min 3 more time(s)...


PM 25
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'faae2345-5020-4dd1-aa4f-e1ac3f6419ba',
    'canceled_at': None,
    'client_order_id': '83c1403d-2405-43bb-b091-f64e4b058be6',
    'created_at': '2020-04-28T16:04:48.395875Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '9c045dc3-dae6-4731-9b6c-b5f7acc3186b',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '25',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-04-28T16:04:48.337854Z',
    'symbol': 'PNW',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-04-28T16:04:48.395875Z'})


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1Min 3 more time(s)...


...

PNW 25
SRE 15
SNPS 13


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1Min 3 more time(s)...



ANSS 7
BRKB
BFB
DUK 22
MKTK
PNW 25


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1Min 3 more time(s)...


SNPS 13
